In [10]:
from dotenv import load_dotenv
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage

from src.lab08.langchain_tools import tools, tool_dict
from src.lab08.langchain_tools import get_current_time, get_yf_stock_history

In [2]:
load_dotenv()  # .env 파일의 api_key 정보를 환경 변수로 로딩.

True

# 한 번에 출력하기

## 언어 모델만 있는 경우 한 번에 출력하기

In [3]:
model = ChatOpenAI(model='gpt-4o-mini')

In [4]:
parser = StrOutputParser()

In [5]:
chain = model | parser

In [6]:
messages = [HumanMessage('한강 작가의 채식주의자를 1000자 이내로 요약해줘.')]

In [7]:
ai_message = chain.invoke(input=messages)

In [8]:
print(ai_message)

한강의 소설 『채식주의자』는 현대 사회에서 개인의 선택과 정체성, 가족과 사회의 갈등을 탐구하는 작품입니다. 이야기는 주인공 영혜가 갑자기 채식주의자가 되면서 시작됩니다. 그녀는 고기를 거부하고 완전히 채식으로 전환하는데, 이는 가족과 주변 사람들에게 큰 충격을 안깁니다.

소설은 세 개의 파트로 나뉘어 있으며, 각 파트는 다른 시점에서 영혜의 삶을 다룹니다. 첫 번째 파트는 남편의 시점에서 영혜의 변화를 바라보는 이야기로, 그녀의 선택이 가정에 미치는 영향을 보여줍니다. 두 번째 파트에서는 영혜의 형부가 등장해 그녀에게 매력을 느끼게 되며, 이로 인해 발생하는 비극적인 사건이 그려집니다. 마지막 파트는 영혜의 언니의 시점으로, 그녀의 복잡한 감정을 통해 가족의 붕괴와 영혜의 고통을 조명합니다.

채식주의가 단순한 식습관의 변화가 아니라 존재에 대한 깊은 질문으로 이어지면서, 작가는 인간의 삶에 대한 심오한 성찰을 담고 있습니다. 이 작품은 변증법적 요소를 통해 인간의 본성과 욕망, 그리고 그것이 사회와 어떻게 얽히는지를 탐구하여 독자에게 강렬한 여운을 남깁니다.


## 도구를 설정한 경우 한 번에 출력하기

In [11]:
# AI 모델과 도구 목록을 바인딩.
model_with_tools = model.bind_tools(tools=tools)

In [12]:
messages = [HumanMessage('서울, 부산, LA의 현재 시간을 비교해줘.')]

In [13]:
ai_message = model_with_tools.invoke(input=messages)

In [14]:
ai_message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  get_current_time (call_p2sCFWBTh29fkB4lCLklL7LA)
 Call ID: call_p2sCFWBTh29fkB4lCLklL7LA
  Args:
    timezone: Asia/Seoul
    location: Seoul
  get_current_time (call_sYkSMMbFBq7A4SNVPyPDHlMG)
 Call ID: call_sYkSMMbFBq7A4SNVPyPDHlMG
  Args:
    timezone: Asia/Seoul
    location: Busan
  get_current_time (call_tmyexjzzPFfbzIFWKIZGUBAe)
 Call ID: call_tmyexjzzPFfbzIFWKIZGUBAe
  Args:
    timezone: America/Los_Angeles
    location: Los Angeles


In [15]:
messages.append(ai_message)  # tool_calls를 가지고 있는 AIMessage를 대화 리스트에 추가!

In [17]:
for tool_call in ai_message.tool_calls:
    print(tool_call['name'], tool_call['args'])
    fn = tool_dict[tool_call['name']]  # tool_dict.get(tool_call['name'])
    tool_msg = fn.invoke(input=tool_call)  # invoke 메서드 호출 -> fn(args) 실행.
    messages.append(tool_msg)

get_current_time {'timezone': 'Asia/Seoul', 'location': 'Seoul'}
get_current_time {'timezone': 'Asia/Seoul', 'location': 'Busan'}
get_current_time {'timezone': 'America/Los_Angeles', 'location': 'Los Angeles'}


In [18]:
for m in messages:
    m.pretty_print()

================================ Human Message =================================

서울, 부산, LA의 현재 시간을 비교해줘.
================================== Ai Message ==================================
Tool Calls:
  get_current_time (call_p2sCFWBTh29fkB4lCLklL7LA)
 Call ID: call_p2sCFWBTh29fkB4lCLklL7LA
  Args:
    timezone: Asia/Seoul
    location: Seoul
  get_current_time (call_sYkSMMbFBq7A4SNVPyPDHlMG)
 Call ID: call_sYkSMMbFBq7A4SNVPyPDHlMG
  Args:
    timezone: Asia/Seoul
    location: Busan
  get_current_time (call_tmyexjzzPFfbzIFWKIZGUBAe)
 Call ID: call_tmyexjzzPFfbzIFWKIZGUBAe
  Args:
    timezone: America/Los_Angeles
    location: Los Angeles
================================= Tool Message =================================
Name: get_current_time

2025-09-16 14:50:34 Asia/Seoul(Seoul)
================================= Tool Message =================================
Name: get_current_time

2025-09-16 14:50:34 Asia/Seoul(Busan)
================================= Tool Message ====================

In [19]:
ai_message = model_with_tools.invoke(input=messages)

In [20]:
ai_message.pretty_print()

================================== Ai Message ==================================

서울과 부산의 현재 시간은 **2025-09-16 14:50:34**입니다.  
반면, LA의 현재 시간은 **2025-09-15 22:50:34**입니다.  

서울과 부산은 동일한 시간을 가지고 있으며, LA는 16시간 뒤처져 있습니다.


# 스트리밍 방식으로 출력하기

## 언어 모델만 있는 경우 스트리밍 출력

In [21]:
messages = [HumanMessage('한강 작가의 채식주의자를 1000자 이내로 요약해줘.')]

In [28]:
response = model.stream(input=messages)

In [29]:
type(response)  #> generator: for-in 구문에서 반복할 때마다 어떤 값을 yield(반환)하는 객체.

generator

In [30]:
for r in response:
    print(r.content, end='')

한강의 소설 '채식주의자'는 현대 사회에서의 인간 존재와 정체성, 그리고 억압을 주제로 다루고 있습니다. 이야기는 주인공 영혜를 중심으로 전개되며, 그녀가 갑자기 채식주의자로 전향하면서 가족과 주변 인물들의 삶에 미치는 영향이 주된 내용입니다.

영혜는 꿈에서 묘사된 잔인한 이미지에 고통받고, 이를 계기로 육식에 대한 반감을 느끼고 채식을 시작합니다. 그녀의 결정은 가족, 특히 남편과 부모에게 큰 충격을 주고, 그로 인해 각 인물은 자신의 강박적인 욕망과 갈등을 드러내게 됩니다. 영혜의 변화는 단순한 식습관의 변화가 아니라, 그녀의 내면 세계와 사회적 규범에 대한 반항을 상징합니다.

각 장은 영혜의 시점에서 진행되며, 그녀의 남편, 여자 형제, 그리고 곁에 있는 인물들이 그녀의 선택에 대한 각기 다른 반응을 보입니다. 이 과정에서 인간의 본성과 욕망, 그리고 상실감이 심도 있게 탐구되어 있습니다.

최종적으로, '채식주의자'는 개인의 자기결정권과 사회적 압박, 그리고 인간 존재의 의미를 깊이 있게 성찰하는 작품입니다. 영혜의 선택은 그녀 자신뿐만 아니라 그녀를 둘러싼 모든 인물들에게도 치명적인 영향을 미치며, 이를 통해 작가는 인간 관계의 복잡성과 불안정함을 드러냅니다.

In [31]:
# model과 parser가 연결된 체인을 사용하는 경우
response = chain.stream(input=[HumanMessage('해리포터와 비밀의 방의 내용을 1000자 이내로 요약해줘. ')])
for r in response:
    print(r, end='')

《해리 포터와 비밀의 방》은 J.K. 롤링의 해리 포터 시리즈 두 번째 책으로, 해리 포터가 호그와트 마법학교의 두 번째 해를 보내는 이야기를 다룹니다. 여름 방학 동안, 해리는 더즐리 가정에서 지내고 있는데, 그들은 그를 무시하고 차별합니다. 해리는 자신의 친구 론 위즐리가 구급차를 타고 그를 구하러 오게 됩니다.

호그와트에 돌아온 해리와 친구들은 학교에서 신비로운 사건들이 발생하는 것을 목격합니다. 학생들과 여러 마법 생물들이 공격받고, '비밀의 방'이 열렸다는 소문이 퍼집니다. 이 방은 뱀파이어의 후손인 슬리데린의 후계자가 숨겨둔 곳으로, 괴물이 풀어져 학교 학생들을 위협하고 있습니다.

해리와 친구들은 진실을 파헤치기 위해 조사를 시작합니다. 해리는 과거의 유령인 모리의 이야기를 듣고, 자신이 뱀의 언어를 구사할 수 있다는 사실을 알게 됩니다. 결국 해리는 소환된 괴물인 바실리스크와 맞서 싸우게 되고, 친구인 진리의 존재가 드러나면서 자신의 정체성과 슬리데린의 후계자에 대한 연결성을 깨닫게 됩니다.

해리는 헬라의 불사조가 자신을 도와줘 바실리스크를 처치하고, 진리와 함께 새로운 친구를 구합니다. 사건이 해결된 후, 해리는 자신의 마법적 능력과 친구들의 소중함을 더욱 깊이 깨닫게 됩니다. 이 이야기는 우정, 용기, 그리고 정체성을 찾는 과정에 대한 중요한 메시지를 전달합니다.

## 도구를 포함하는 모델의 스트리밍 출력